# Masking

In [1]:
import numpy as np
import scipp as sc

## Creating and manipulating masks

Masks are simply variables with `dtype=bool`:

In [2]:
mask = sc.Variable(['x'], values=[False, False, True])
sc.table(mask)

Boolean operators can be used to manipulate such variables:

In [3]:
print(~mask)
print(mask ^ mask)
print(mask & ~mask)
print(mask | ~mask)

    <scipp.Variable>          bool       [dimensionless]  (x)  [True, True, False]

    <scipp.Variable>          bool       [dimensionless]  (x)  [False, False, False]

    <scipp.Variable>          bool       [dimensionless]  (x)  [False, False, False]

    <scipp.Variable>          bool       [dimensionless]  (x)  [True, True, True]



## Masks in DataArray and Dataset

Data arrays and dataset can store arbitrary masks.
This is accessible using the `masks` keyword-argument and property, which behaves in the same way as `coords` and `attrs`:

In [4]:
a = sc.DataArray(
    data = sc.Variable(dims=['y', 'x'], values=np.arange(1.0, 7.0).reshape((2, 3))),
    coords={
        'y': sc.Variable(['y'], values=np.arange(2.0), unit=sc.units.m),
        'x': sc.Variable(['x'], values=np.arange(3.0), unit=sc.units.m)},
    masks={
        'x': sc.Variable(['x'], values=[False, False, True])}
    )
sc.show(a)

In [5]:
b = a.copy()
b.masks['x'].values[1] = True
b.masks['y'] = sc.Variable(['y'], values=[False, True])

Note that setting a mask does *not* affect the data.

## Operations with masked objects

### Element-wise binary operations

The result of operations between data arrays or dataset with masks contains the masks of both inputs.
If both inputs contain a mask with the same name, the output mask is the combination of the input masks with an  **OR** operation:

In [6]:
a + b

<scipp.DataArray>
Dimensions: {{y, 2}, {x, 3}}
Coordinates:
    x                         float64    [m]              (x)  [0.000000, 1.000000, 2.000000]
    y                         float64    [m]              (y)  [0.000000, 1.000000]
Masks:
    x                         bool       [dimensionless]  (x)  [False, True, True]
    y                         bool       [dimensionless]  (y)  [False, True]
Data:
                              float64    [dimensionless]  (y, x)  [2.000000, 4.000000, ..., 10.000000, 12.000000]

### Reduction operations

Operations like `sum` and `mean` over a particular dimension cannot preserve masks that depend on this dimension.
If this is the case, the mask is applied during the operation and is not present in the output:

In [7]:
sc.sum(a, 'x')

<scipp.DataArray>
Dimensions: {{y, 2}}
Coordinates:
    y                         float64    [m]              (y)  [0.000000, 1.000000]
Data:
                              float64    [dimensionless]  (y)  [3.000000, 9.000000]

The `mean` operation takes into account that masking is reducing the number of points in the mean, i.e., masked elements are not counted (in contrast to, e.g., treating them as 0):

In [8]:
sc.mean(a, 'x')

<scipp.DataArray>
Dimensions: {{y, 2}}
Coordinates:
    y                         float64    [m]              (y)  [0.000000, 1.000000]
Data:
                              float64    [dimensionless]  (y)  [1.500000, 4.500000]

If a mask does not depend on the dimension used for the `sum` or `mean` operation, it is preserved.
Here `b` has two masks, one that is applied and one that is preserved:

In [9]:
sc.sum(b, 'x')

<scipp.DataArray>
Dimensions: {{y, 2}}
Coordinates:
    y                         float64    [m]              (y)  [0.000000, 1.000000]
Masks:
    y                         bool       [dimensionless]  (y)  [False, True]
Data:
                              float64    [dimensionless]  (y)  [1.000000, 4.000000]